## Prepare ElasticSearch and Ollama

# Prepare ElasticSearch and OpenAI API

# Prepare ElasticSearch and OpenAI API 

In [1]:
# !pip install openai elasticsearch
from openai import OpenAI
from elasticsearch import Elasticsearch

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [3]:
# Increase timeout settings
es_client = Elasticsearch('http://localhost:9200', 
                          request_timeout=60, 
                          max_retries=10, 
                          retry_on_timeout=True)

In [4]:
# Check cluster health
health = es_client.cluster.health()
print("Cluster Health:", health)

Cluster Health: {'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 2, 'active_shards': 2, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [5]:
# Create index 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

In [6]:
# Delete the existing index if it exists
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print(f"Deleted existing index: {index_name}")

# Create the new index
es_client.indices.create(index=index_name, 
                         body=index_settings)
print(f"Created new index: {index_name}")

Deleted existing index: course-questions
Created new index: course-questions


In [7]:
import requests 
from tqdm.auto import tqdm

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
# Index all documents
for doc in tqdm(documents):
    es_client.index(index = index_name, 
                    body = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Download the documents

In [9]:
# verify 
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# RAG pipeline 

# RAG Pipeline

In [10]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", 
                                   "text", 
                                   "section"],
                        "type": "best_fields"
                    }
                },
                # this is optional to filter e.g. for specific courses 
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

    response = es_client.search(index=index_name, 
                                body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
    
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'phi3',
        messages = [{                 
            "role": "user", 
            "content": prompt
        }])

    return response.choices[0].message.content

In [13]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    print(llm(prompt)) 



In [16]:
# query = "How do I run Kafka?"
# query = "What do we learn about AWS Lambda?"
query = "The has already started. Can I still join?"
rag(query)